[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io/)

# AI Program

## Exercise 0008 - Deep Learning - Convolution NN for Image Classification

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 20/05/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/Exercise0008.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning


# Deep Learning
import torch
import torch.nn            as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torchinfo
from torchmetrics.classification import BinaryAccuracy
import torchvision
from torchvision.transforms import v2 as TorchVisionTrns

# Miscellaneous
import copy
import gdown
import json
import os
import random
import re
import urllib.request
import shutil
import zipfile

# Typing
from typing import Any, Callable, Dict, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())


In [ ]:
# Constants

DATA_FOLDER_PATH = 'Data'

# Data Set Links: 
# - MicroSoft Kaggle Cats and Dogs Dataset - https://www.microsoft.com/en-us/download/details.aspx?id=54765 (Dog 11702, Cat 666 might be corrupted)
# - Kaggle Competition - Dogs vs. Cats - https://www.kaggle.com/c/dogs-vs-cats
DATA_SET_URL            = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip'
DATA_SET_FILE_NAME      = 'CatsDogs.zip'
DATA_SET_FOLDER_NAME    = 'CatsDogs'

D_CLASSES  = {0: 'Cat', 1: 'Dog'}
L_CLASSES  = ['Cat', 'Dog']


In [ ]:
# Course Packages

from DataManipulation import DownloadUrl
from DeepLearningPyTorch import TrainModel


In [ ]:
# General Auxiliary Functions

# Class to handle a folder with no labels: Test

from torchvision.datasets.folder import IMG_EXTENSIONS, pil_loader

class TestDataSet( torchvision.datasets.VisionDataset ):
    def __init__(self, root: str = None, transforms: Callable[..., Any] | None = None, transform: Callable[..., Any] | None = None, target_transform: Callable[..., Any] | None = None) -> None:
        super().__init__(root, transforms, transform, target_transform)


        lF = os.listdir(root)
        lFiles = [fileName for fileName in lF if (os.path.isfile(os.path.join(root, fileName)) and (os.path.splitext(os.path.join(root, fileName))[1] in IMG_EXTENSIONS))]

        self.lFiles = lFiles
        self.loader = pil_loader
    
    def __len__(self) -> int:
        
        return len(self.lFiles)
    
    def __getitem__(self, index: int) -> Any:
        
        imgSample =  self.loader(os.path.join(self.root, self.lFiles[index]))
        if self.transform is not None:
            imgSample = self.transform(imgSample)
        
        return imgSample


## Exercise: Cats vs. Dogs

This exercises builds a model based on _Convolutional Neural Network_ for _Binary Image Classification_.  
The data set is from the [Kaggle - Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats) competition.  
The objective is to classify an image either as a _Cat_ or a _Dog_.  

The challenge in this data set is working with images with different dimensions.

The data contains 25,000 RGB images with different dimensions.  

Tasks:
 - Download and arrange data properly.
 - Split data into 22,500 train samples and 2,500 validation samples.
 - Build a dataset and data loader.  
   The data loader must support the case of different image dimensions.
 - Build a parameterized model (Layers, Activations, etc...).
 - Build an optimizer and a scheduler.
 - Build a training loop to optimize hyper parameters.

Tips:
 - Use random transformation to enrich the data set.  
   See [`torchvision.transforms.RandomRotation`](https://pytorch.org/vision/main/generated/torchvision.transforms.RandomRotation.html) as an examples.  
   This is called [_Data Augmentation_](https://en.wikipedia.org/wiki/Data_augmentation).
 - Use [`torchvision.datasets.ImageFolder`](https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html) to load the data easily.
 - Use [`torch.utils.data.random_split`](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) to split the data set.
 - You may handle the different image dimensions by:
    - Build the model in a manner which is dimension insensitive.
    - Transform the image into a pre defined size (Use padding to keep aspect ratio).
 - 

**Objective**: Above 96% accuracy on the validation set (See the [Competition Leader Board](https://www.kaggle.com/c/dogs-vs-cats/leaderboard)).

* <font color='brown'>(**#**)</font> One may use single output with the Binary Cross Entropy Loss: [`BCELoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html), [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html).

### Handling Different Images Size

There are many options to handle different images size in the same batch / data.  
There are 2 main approaches:

1. Build a Model Insensitive to Image Input  
   Build a model which assumes no knowledge of the image size.  
   Usually it is built by Convolution Layers and Adaptive Pooling so the output is a function of the known number of channels and not the input.  
   The challenge is to handle loading and processing the data.  
   It usually done by padding all images to the size of the largest image.
2. Adapt the Data  
   Apply some combination of padding, resize and crop to have the same image size.

* <font color='brown'>(**#**)</font> For FCN models (1) one could also set the batch size to 1 so each image is on its own. Yet it hurts efficiency greatly.
* <font color='brown'>(**#**)</font> See [PyTorch Forum - How to Create a `dataloader` with Variable Size Input](https://discuss.pytorch.org/t/8278).
* <font color='brown'>(**#**)</font> See [StackOverflow - How to Train Network on Images of Different Sizes with PyTorch](https://stackoverflow.com/questions/72595995).


In [ ]:
# Parameters




## Load Data

Load the classification data set.

1. Download the Data Set from [Kaggle Competition - Cat vs. Dog Dataset](https://www.kaggle.com/c/dogs-vs-cats).
2. There are pre defined train and test (Not labeled) split.  
3. Build a script to create a folder `Validation` and move to it part of the labeled data.